In [3]:
# Importing the library
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, gc
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score, learning_curve
import fasttext
import csv
import nltk
from nltk.corpus import stopwords
# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

In [7]:
string = "overheating"
def fasttext_potential_acclvl(string):
    ds = pd.read_csv('industrial_safety_and_health_database_with_accidents_description.csv')
    ds.rename(columns= {'Data':'Date', 'Countries':'Country', 'Genre':'Gender',
                    'Employee or Third Party':'Employee type'}, inplace =True)
    ds['Date'] = pd.to_datetime(ds['Date'])
    ds['Year'] = ds['Date'].apply(lambda x: x.year)
    ds['Month'] = ds['Date'].apply(lambda x: x.month)
    ds['Day'] = ds['Date'].apply(lambda x: x.day)
    ds['Weekday'] = ds['Date'].apply(lambda x: x.day_name())
    def month_quater_conversion(x):
      if x in [1, 2, 3]:
        season = 'First'
      elif x in [4, 5, 6]:
        season = 'Second'
      elif x in [7, 8, 9]:
        season = 'Third'
      elif x in [10, 11, 12]:
        season = 'Fourth'
      return season
    ds['Quater'] = ds['Month'].apply(month_quater_conversion)
    # Defining the level as per Accident Level
    def Accident_level_conversion(x):
      if x in ['I','II']:
        level = 'Low'
      elif x in ['III']:
        level = 'Medium'
      elif x in ['IV', 'V', 'VI']:
        level = 'High'
      return level
    ds['Accident Severity'] = ds['Accident Level'].apply(Accident_level_conversion)
    ds['Potential Severity'] = ds['Potential Accident Level'].apply(Accident_level_conversion)
    replace_value = {'Local_01':1, 'Local_02':2, 'Local_03':3, 'Local_04':4, 'Local_05':5, 'Local_06':6, 'Local_07':7, 'Local_08':8, 
                 'Local_10':10, 'Local_09':9, 'Local_11':11, 'Local_12':12}
    ds['Local'] = ds['Local'].map(replace_value)

    replace_value = {'I':1, 'IV':4, 'III':3, 'II':2, 'V':5}
    ds['Accident Level'] = ds['Accident Level'].map(replace_value)

    replace_value = {'IV':4, 'III':3, 'I':1, 'II':2, 'V':5, 'VI':6}
    ds['Potential Accident Level'] = ds['Potential Accident Level'].map(replace_value)

    del replace_value
    
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stopwords = set(stopwords.words('english'))
    ds['clean_Description'] = ds['Description'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))    #Fasttext

    ft_df_Potential = pd.DataFrame(columns=['fasttext_data'])
    ft_df_Potential['fasttext_data'] ='__label__' + ds['Potential Accident Level'].astype(str) + " "+ds['clean_Description']   
    
    ft_df_Potential.to_csv(r'fasttext_industrial_Potential.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    !head -n 340 fasttext_industrial_Potential.txt > ft_ind_Potential.train
    !tail -n 85 fasttext_industrial_Potential.txt > ft_ind_Potential.valid
    ft_model_Potential = fasttext.train_supervised(input="ft_ind_Potential.train")
    
    train_Potential = ft_df_Potential.head(340)
    valid_Potential = ft_df_Potential.tail(85)
    train_Potential.to_csv(r'train_Potential.train', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    valid_Potential.to_csv(r'valid_Potential.valid', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
    train_Potential = open("train_Potential.txt","a")
    valid_Potential = open("valid_Potential.txt","a")
    
    ft_model_Potential = fasttext.train_supervised(input="train_Potential.train")
    
    ft_model_Potential.save_model("ft_model_Potential.bin")
    pred = ft_model_Potential.predict(string) #@@@###@@@#@@@#@@@
    print(pred)
    return pred

a = fasttext_potential_acclvl(string)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\reach\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(('__label__4',), array([0.16707644]))
